<h1>Анализ прибыльности региона для добычи нефти</h1>

Допустим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.
Шаги для выбора локации обычно такие:

    В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
    Строят модель для предсказания объёма запасов в новых скважинах;
    Выбирают скважины с самыми высокими оценками значений;
    Определяют регион с максимальной суммарной прибылью отобранных скважин.

Нам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Необходимо построить модель для определения региона, где добыча принесёт наибольшую прибыль, и проанализировать возможную прибыль и риски техникой Bootstrap.

**Условия задачи:**

    Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
    При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
    Бюджет на разработку скважин в регионе — 10 млрд рублей.
    При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
    После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

**Описание данных:**

Данные геологоразведки трёх регионов находятся в файлах: geo_data_0.csv, geo_data_1.csv, geo_data_2.csv.

    id — уникальный идентификатор скважины;
    f0, f1, f2 — три признака точек;
    product — объём запасов в скважине (тыс. баррелей).

**План работы**: первым делом мы ознакомимся с полученными данными, проведем подготовку признаков, если потребуется, затем - обучим по модели, предсказывающей объем запасов в скважине, для каждого региона, проверим качество и сделаем предсказание, после этого мы используем bootstrap для оценки прибыли и рисков в каждом регионе, сделаем выводы.

<h3>Оглавление</h3>

1. [Шаг 1: Знакомство с данными и подготовка признаков](#start)
2. [Шаг 2: Обучение и проверка моделей](#model)
3. [Шаг 3: Подготовка к расчету прибыли и рисков](#prepare)
4. [Шаг 4: Расчет прибыли и рисков: выводы](#risks)

<h3>Шаг 1: Знакомство с данными и подготовка признаков</h3>
<a id='start'></a>

Первым делом импортируем нужные нам библиотеки.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from numpy.random import RandomState

Загрузим данные.

In [2]:
data1 = pd.read_csv('/datasets/geo_data_0.csv')
data2 = pd.read_csv('/datasets/geo_data_1.csv')
data3 = pd.read_csv('/datasets/geo_data_2.csv')

Итак, у нас по датасету на регион. Познакомимся с ними поближе - для каждого датасета посмотрим его первые пять строк, общую информацию, а также - на статистики столбцов.

In [3]:
def meeting(data):
    print(data.head())
    print(' ')
    print(data.info())
    print(' ')
    print(data.describe())

Первый.

In [4]:
meeting(data1)

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
 
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580

Второй.

In [5]:
meeting(data2)

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
 
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%     

Третий.

In [6]:
meeting(data3)

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
 
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288

Что можно сказать? 

    - в данных отсутствуют пропуски в явном виде; 
    - типы столбцов во всех трех датасетах (кроме id) соответствуют требуемым; 
    - отсутствуют явные маркеры наличия выбросов, то есть, такие величины как: mean, min, max и std согласуются между собой; 
    - признаки в рамках датасета, а также - один и тот же признак в разных датасетах не имеют сильных различий в диапазонах значений, которые они принимают; 

Мы ознакомились с данными. В рамках подготовки признаков следует лишь удалить столбец с id. А также - отделить целевой признак.

In [7]:
data1 = data1.drop('id', axis=1)
data2 = data2.drop('id', axis=1)
data3 = data3.drop('id', axis=1)

In [8]:
data1_target = data1['product']
data2_target = data2['product']
data3_target = data3['product']
data1_features = data1.drop('product', axis=1)
data2_features = data2.drop('product', axis=1)
data3_features = data3.drop('product', axis=1)

Получили 6 датафреймов: по 2 на каждый регион - признаки и целевой признак.

Дальнейшая подготовка признаков не требуется.

**Вывод по Шагу 1**: на данном шаге мы познакомились с поступившими данными, не выявили в них ничего, что требует дополнительного внимания; в рамках подготовки признаков мы удалили id-столбец, а также отделили целевой признак в отдельный датафрейм.

<h3>Шаг 2: Обучение и проверка моделей</h3>
<a id='model'></a>

На данном этапе нам предстоит обучить 3 модели методом линейной регрессии: по одной модели на регион. Для каждой модели у нас своя таблица с признаками и целевым признаком. После обучения необходимо будет совершить некоторые предсказания, а также - измерить качество моделей.

**Модель предсказания объемов скважины для первого региона**

Итак, признаки хранятся в data1_features, целевой признак - в data1_target. Поделим данные наборы на тренировочную и валидационную выборки в пропорции 3:1.Создадим и обучим модель. Для этих действий напишем функцию, которая будет возвращать нам предсказания модели и валидационную часть целевого признака - тоже потребуется.

In [9]:
def split_train(features, target):
    feat_train, feat_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25,
                                                                                           random_state=12345)
    model = LinearRegression()
    model.fit(feat_train, target_train)
    predictions = model.predict(feat_valid)
    return target_valid, predictions

In [10]:
data1_target_valid, predictions1 = split_train(data1_features, data1_target)

Совершим предсказания. Сохраним их вместе с индексами признаков, для которых предсказывали.

In [11]:
predicted1 = pd.DataFrame()
predicted1['index'] = data1_target_valid.index
predicted1['pred'] = predictions1

Посмотрим на средний запас предсказанного сырья, на реальный средний запас, а также на RMSE и R2 модели.

In [12]:
def model_info(target_valid, predicted):
    print('Средний запас реального сырья: {:.4} тыс. баррелей'.format(target_valid.mean()))
    print('Средний запас предсказанного сырья: {:.4} тыс. баррелей'.format(predicted['pred'].mean()))
    mse1 = mean_squared_error(target_valid, predicted['pred'])
    rmse1 = mse1 ** 0.5
    print('RMSE данной модели: {:.4} тыс. баррелей'.format(rmse1))
    r2 = r2_score(target_valid, predicted['pred'])
    print('R2 данной модели: {:.4}'.format(r2))

In [13]:
model_info(data1_target_valid, predicted1)

Средний запас реального сырья: 92.08 тыс. баррелей
Средний запас предсказанного сырья: 92.59 тыс. баррелей
RMSE данной модели: 37.58 тыс. баррелей
R2 данной модели: 0.2799


Какой можно сделать вывод? 

Итак, средние предсказанных значений и реальных значений почти совпадают. Корень среднего квадратичного отклонения составляет около 38 тыс. баррелей, что весьма немало на фоне среднего значения предсказанного набора, но не так это и много, как могло бы быть - доказательство тому - значение метрики r2. R2 положительно, более того - это почти треть от единицы, то есть модель работает лучше случайной.

Построим еще две модели - для двух оставшихся регионов. Рассмотрим те же их характеристики, что и для этой модели.

**Модель предсказания объемов скважины для второго региона**

Поделим выборки. Создадим и обучим модель.

In [14]:
data2_target_valid, predictions2 = split_train(data2_features, data2_target)

Совершим предсказания.

In [15]:
predicted2 = pd.DataFrame()
predicted2['index'] = data2_target_valid.index
predicted2['pred'] = predictions2

Теперь качество.

In [16]:
model_info(data2_target_valid, predicted2)

Средний запас реального сырья: 68.72 тыс. баррелей
Средний запас предсказанного сырья: 68.73 тыс. баррелей
RMSE данной модели: 0.8931 тыс. баррелей
R2 данной модели: 0.9996


Сделаем вывод: получилась почти идеальная модель - средние предсказанных значений и истинных почти совпадают; значение RMSE очень мало, что особенно важно выглядит при взгляде на R2 - значение метрики практически равно единице. Чтобы выяснить, с чем это может быть связано - посмотрим на коэффициенты корреляции между признаками и целевым признаком.

In [17]:
data2.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


Получили интересный результат - коэффициент корреляции между признаком f2 и целевым признаком практически равен единице, это значит, что существует сильная линейная зависимость, которую было относительно легко распознать, поэтому модель и получилась такой хорошей.

**Модель предсказания объемов скважины для третьего региона**

Делим выборки.Создадим и обучим модель. Сделаем предсказание.

In [18]:
data3_target_valid, predictions3 = split_train(data3_features, data3_target)
predicted3 = pd.DataFrame()
predicted3['index'] = data3_target_valid.index
predicted3['pred'] = predictions3

Посмотрим на качество.

In [19]:
model_info(data3_target_valid, predicted3)

Средний запас реального сырья: 94.88 тыс. баррелей
Средний запас предсказанного сырья: 94.97 тыс. баррелей
RMSE данной модели: 40.03 тыс. баррелей
R2 данной модели: 0.2052


Сделаем вывод. Как и в ситуации с двумя предыдущими моделями - средние почти совпадают. Особенность данной модели: наибольшее значение RMSE - очень большие отклонения у предсказаний от истинных значений. R2 сообщает то же самое - ее значение не велико, но есть и позитивная сторона - r2 вообще больше нуля, причем на 0.2 - это неплохо. Хотя на фоне предыдущей - невероятно удачной модели - выглядит грустно.

**Вывод по Шагу 2**: на данном Шаге мы создали и обучили 3 модели - по модели на регион. Также мы посмотрели на качество этих моделей: получили две средних и одну очень удачную. Сохранили предсказания моделей в отдельных переменные, с которыми мы потом будем работать для оценки рисков и прибыли.

<h3>Шаг 3: Подготовка к расчету прибыли и рисков</h3>
<a id='prepare'></a>

Итак. У нас есть предсказания по объему нефти в скважинах. У нас также есть знания о: стоимости каждой тысячи баррелей из этих скважин (450 тыс. рублей), о бюджете на разработку скважин в одном регионе. Также мы знаем, сколько скважин разрабатывается в каждом выбранном регионе - 200 лучших из 500 рассмотренных. 

Посчитаем, во сколько в среднем обходится одна скважина в выбранном регионе.

Занесем значение 10 млрд. рублей в переменную - все вычисления далее будут вестись в тыс. рублей.

In [20]:
BUDGET = 10000000
BEST_HOLES = 200
HOLES = 500
budget_per_sq = BUDGET / 200
budget_per_sq

50000.0

Сколько тыс. баррелей должна принести эта скважина, чтобы окупить вложенные в нее деньги и начать приносить доход? 

In [21]:
DOH = 450 
barrels = budget_per_sq / DOH
round(barrels,0)

111.0

Получили 111 тыс.баррелей. Вспомним наши средние значения содержания продукта в скважине для каждого региона. 

In [22]:
print('Среднее содержание в скважине в регионе 1: {:.3}'.format(data1_target_valid.mean()))
print('Среднее содержание в скважине в регионе 2: {:.3}'.format(data2_target_valid.mean()))
print('Среднее содержание в скважине в регионе 3: {:.3}'.format(data3_target_valid.mean()))

Среднее содержание в скважине в регионе 1: 92.1
Среднее содержание в скважине в регионе 2: 68.7
Среднее содержание в скважине в регионе 3: 94.9


Какой можно сделать вывод? Ни в одном регионе среднее значение не дотягивает до требуемого. Особенно по втором регионе. Но, как мы знаем, из 500 исследуемых скважин берется в работу только 200 лучших. Возможно, данный факт спасет ситуацию.

**Выводу по Шагу 3**: итак, на данном шаге мы рассмотрели вопрос - сколько тыс. баррелей должна выдать скважина, чтобы окупиться, результат получили весьма печальный - значение высокое. 

<h3>Шаг 4: Расчет прибыли и рисков: выводы</h3>
<a id='risks'></a>

Итак, нам осталось посчитать риски и прибыль для каждого региона с помощью техники Bootstrap: мы будем брать 1000 выборок по 500 скважин из предсказанного набора, считать прибыль с этой выборки (а точнее - с ее лучших двухсот скважин) путем суммирования соответствующих значений из валидационной выборки, так мы получим распределение прибыли. 

Получив распределение, найдем среднюю прибыль, 95%-й доверительный интервал. Также оценим риски.

Напишем функцию, которая считает прибыль с выборки из пятисот скважин. Функция выбирает лучшие 200 скважин и считает, сколько тыс. баррелей соответствующие скважины из валидационного набора принесут суммарно, сколько это стоит денег и сколько это прибыли.

In [23]:
def revenue(sq_list, real): 
    global DOH, BUDGET
    best = sq_list.sort_values(by='pred', ascending=False)[:BEST_HOLES]
    best_sum = 0
    for i in best['index']:
        best_sum += real[i]
    oborot = best_sum *  DOH
    return oborot - BUDGET

Приступим к бутстрепу и дальнейшему анализу.

**Первый регион**

Будем сохранять прибыль по каждой выборке в списке profit.

In [24]:
state = RandomState(12345)
def bootstrap(predicted, real):
    profit = []
    for i in range(1000):
        samp = predicted.sample(n=HOLES, random_state=state, replace=True)
        profit_1 = revenue(samp, real)
        profit.append(profit_1)
    return profit

In [25]:
profit1 = bootstrap(predicted1, data1_target_valid)

Посмотрим на среднее значение, на 95-процентный доверительный интервал. Также - оценим риски. Для этого нам надо найти на каком квантиле заканчиваются отрицательные значения. 

In [26]:
def profit_desc(data):
    new_data = pd.Series(data)
    print('Среднее значение прибыли: {:.2f} тыс.рублей'.format(new_data.mean()))
    print('95-процентный доверительный интервал: от {:.2f} тыс. рублей до {:.2f} тыс. рублей'.format(
        new_data.quantile(0.025), new_data.quantile(0.975)))
    less_then_0_proc = round(len(new_data[new_data<0])/len(new_data), 3) * 100
    print('Вероятность убытков: {:.3f} %'.format(less_then_0_proc))

In [27]:
profit_desc(profit1)

Среднее значение прибыли: 396164.98 тыс.рублей
95-процентный доверительный интервал: от -111215.55 тыс. рублей до 909766.94 тыс. рублей
Вероятность убытков: 6.900 %


Что можно сказать по полученному результату?

Первое, что бросается в глаза - высокий уровень рисков. Он выше допустимого (2,5 %), поэтому данный регион не может быть выбран как регион для развития в нем скважин. Хотя среднее значение достаточно высоко - 396 млн рублей. 

**Второй регион**

In [28]:
profit2 = bootstrap(predicted2, data2_target_valid)
profit_desc(profit2)

Среднее значение прибыли: 461155.82 тыс.рублей
95-процентный доверительный интервал: от 78050.81 тыс. рублей до 862952.06 тыс. рублей
Вероятность убытков: 0.700 %


По второму региону вывод более утешительный - вероятность убытков в пределах нормы. Среднее значение выше, чем в первом регионе, и составляет 461 млн рублей. 

**Третий регион**

In [29]:
profit3 = bootstrap(predicted3, data3_target_valid)
profit_desc(profit3)

Среднее значение прибыли: 392950.48 тыс.рублей
95-процентный доверительный интервал: от -112227.63 тыс. рублей до 934562.91 тыс. рублей
Вероятность убытков: 6.500 %


Опять получили высокую вероятность убытков. Этот регион тоже выбывает из голосования. Хотя прибыль в таком случае может достигать аж 0,93 млрд рублей. 

**Вывод по Шагу 4:** итак, на данном Шаге мы построили распределение прибыли для каждого из трех регионов с помощью техники bootstrap. Посчитали средние значения прибыли, построили доверительные интервалы и оценили риски. Ответ на вопрос "Какой же регион выбрать в итоге?" получился сам: по двум из трех мы просто получили неудовлетворительные результаты по рискам, поэтому мы можем взять только оставшийся - регион 2. При этом его показатели - среднее значение прибыли и доверительные интервал обнадеживают, прибыль может достигать 863 млн рублей. И вероятность рисков небольшая - всего 1.5 %. Выбор однозначно падает на этого кандидата.